#### Notebook with the functions of the different forecasting models

This notebook contains the functions that allow to define and train forecasting models using the available algorithms,
which are:

  * SARIMAX
  * Prophet
  * LightGBM

The functions included are:

| Function | Description |
| -------- | ----------- |
| `obtain_sarimax`  | defines and trains a SARIMAX model using the given hyperparameters and input time series |
| `obtain_prophet`  | defines and trains a Prophet model using the given hyperparameters, this can be univariate or multivariate |
| `obtain_lightgbm`  | TBD |

###### Definition of functions

In [ ]:
def obtain_sarimax(df_train, params, holidays=False):
    """
    Defines and trains a SARIMAX model according to the specified parameters and input time series.

    If "holidays" is set to True then it is assumed that "df_train" includes a binary column which indicates the
    observations that correspond to holidays, this column will be used as an external regressor.

    Parameters
    __________
        df_train (pd.DataFrame): Dataset with training time series.
        params (dict): Dictionary with seasonal and non-seasonal order parameters of the model.
        holidays (bool, defaults to False): Flag to indicate whether the dataset contains the holidays regressor or not.

    Returns
    _______
        model (ARIMAResultsWrapper): Object with the SARIMAX model defined by "params" and trained with the given time
            series.
    """
    # Validating holidays column
    if holidays:
        exog = df_train["holiday"]
    else:
        exog = None

    # Defining and training SARIMAX model
    model = ARIMA(
        df_train["y"],
        exog=exog,
        order=(params["p"], params["d"], params["q"]),
        seasonal_order=(params["P"], params["D"], params["Q"], params["s"]),
        enforce_invertibility=False,
        enforce_stationarity=False
    ).fit()

    return model

In [ ]:
def obtain_prophet(df_train, params, df_holidays=None):
    """
    Defines and trains a Prophet model according to the specified parameters and input time series.

    If "df_holidays" is different from "None" then it is assumed that this variable contains a DataFrame with the
    holidays in the format Prophet expects them to be.

    It is necessary to consider that Prophet requires the date column to be labeled as "ds" and the series column to be
    labeled as "y". The same naming convention is required for the DataFrame with the holidays as well, with a column
    named "holiday" which must contain the name of the holiday.

    Parameters
    __________
        df_train (pd.DataFrame): Dataset with training time series.
        params (dict): Dictionary with the hyperparameters of the model.
        df_holidays (pd.DataFrame, defaults to None): Dataset with holidays.

    Returns
    _______
        model (prophet.forecaster.Prophet): Object with the Prophet model defined by "params" and trained with the given
            time series.
    """
    # Defining and training Prophet model
    model = Prophet(
        changepoint_range=0.9,
        daily_seasonality=False,
        holidays=df_holidays,
        changepoint_prior_scale=params["changepoint_prior_scale"],|
        seasonality_prior_scale=params["seasonality_prior_scale"],
        holidays_prior_scale=params["holidays_prior_scale"],
    ).fit(df_train)

    return model
